# Advanced RAG란 무엇인가

**RAG**는 LLM(대규모 언어 모델)이 답을 만들 때, **외부 지식(문서/DB/웹 등)을 검색(Retrieval)해서 근거(Context)로 붙인 뒤 생성(Generation)**하는 구조이다. 그런데 기본 RAG(Naive RAG)는 실무에서 다음 문제가 있다.

- 검색이 "비슷한 것"은 찾는데 "정답 근거"를 제대로 못 찾는 경우
- 근거가 있어도 LLM이 **hallucination(환각)**를 섞거나, 근거와 다른 답을 하는 경우
- 검색한 문서가 너무 길고 복잡해서 **정답에 필요한 부분**(**chunk**)을 LLM이 제대로 찾지 못해 답변 품질이 안좋은 경우

**Advanced RAG**는 이런 "현업형 문제"를 줄이기 위해 **데이터베이스 구축 단계, 검색 전단계, 검색 후 단계, 생성 단계에서 고도화**하는 설계 패턴이다.
기본 RAG(Naive RAG)가 "사람이 대충 찾아준 참고자료로 글 쓰는 것"이라면, Advanced RAG는 "사서(검색) + 편집자(정제) + 팩트체커(검증) + 작가(생성)가 협업하는 파이프라인"에 가깝다.

## 종류

### 검색 품질을 올리는 고급 Retrieval

- **하이브리드 검색**: 키워드(BM25) + 벡터(임베딩) 조합
- **멀티-쿼리/쿼리 재작성(Query rewriting)**: 질문을 더 잘 검색되게 바꿔 검색 성공률을 올림
- **메타데이터 필터링**: 메타데이터 필터링을 통해 범위를 좁혀 **정확도**를 올림. 의미기반 검색과 키워드 검색을 조합한 효과.
- **리랭킹(Re-ranking)**: 후보 문서를 많이 가져온 뒤, "정답에 가까운 순서"로 다시 정렬

### Chunking/인덱싱 전략 고도화
- 벡터 데이터 베이스 구축시 어떤 구조로 문서를 분할하고 Indexing 할지의 전략
  - **구조 기반 분할**: 헤더/섹션/표/코드블록
  - **계층형 인덱스**: 문서 요약 → 섹션 → 세부 chunk, Parent-Child 구조
  - **윈도우 확장**: 필요 시 앞 뒤 문맥을 함께 붙임
  
### 검색 결과의 "정제/조립" 단계 추가

- 중복 제거, 노이즈 제거, 관련 부분만 발췌
- 여러 chunk를 **질문 관점으로 재구성**
  → LLM에 그대로 던지는 게 아니라 "답변에 쓰기 좋은 근거 묶음"으로 편집하여 전달한다.

### 생성 단계의 신뢰성 강화(가드레일)

- **근거 기반 답변 강제**: 검색한 문서에 답변의 근거가 없으면 "답을 모른다."고 답변하도록 프롬프트를 구성한다.
- **인용/근거 스니펫 포함**: 어느 문서에서 답이 나왔는지 근거를 보여주도록 하여 답변의 신뢰성을 높인다.
- **자기검증(Verification)**: 답을 만든 뒤 "근거와 모순 없는지" 재확인 하도록 한다.
- **불확실성 처리**: 애매하면 추가 질문(Clarification)을 하거나 또는 답변 범위 제한한다.


## Advanced RAG가 필요한 일반적인 경우

- 문서가 많고(수천~수백만), 구조가 복잡한 경우(매뉴얼/규정/기술문서)
- 질문이 단순 검색이 아니라 "비교/조건/절차/근거 요구"가 많은 복잡한 질문인 경우.
- 최신성/정확성이 중요한 경우(정책, 금융, 의료, 보안, 운영 장애 대응)
- "대충 그럴듯한 답"이 아니라 **근거가 있는 답**이 필수인 경우


In [1]:
###############################################################
# VectorStore, Retriever 준비
###############################################################
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI
from langchain_qdrant import FastEmbedSparse, QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, SparseVectorParams, VectorParams
from langchain_openai import OpenAIEmbeddings

In [3]:
# ##############################################################
# 데이터 준비
##############################################################

def load_and_split_olympic_data(file_path="data/olympic_wiki.md"):
    with open(file_path, "r", encoding="utf-8") as fr:
        olympic_text = fr.read()

    # Split
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ],
        # strip_headers=False, # 문서에 header 포함 여부(default: True - 제거)
    )

    return splitter.split_text(olympic_text)

In [4]:
#################################################################
# Vector DB 연결
# retriever 생성
#################################################################

# collection 삭제후 생성 (데이터 넣지는 않음)
def get_vectorstore(collection_name: str = "olympic_info_wiki"):

    #######################################
    # Qdrant Collection 생성 (sparse + dense)
    #######################################
    dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
    sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

    client = QdrantClient(host="localhost", port=6333)

    #삭제후 생성
    if client.collection_exists(collection_name):
        result = client.delete_collection(collection_name=collection_name)

    client.create_collection(
        collection_name=collection_name,
        vectors_config={"dense": VectorParams(size=3072, distance=Distance.COSINE)},
        sparse_vectors_config={ 
            "sparse": SparseVectorParams()
        },
    )

    ######################################
    # VectorStore 생성 (Hybrid 모드)
    ######################################
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
    
        embedding=dense_embeddings,
        
        sparse_embedding=sparse_embeddings,
        retrieval_mode=RetrievalMode.HYBRID,
    
        vector_name="dense",
        sparse_vector_name="sparse",
    )
    
    ######################################
    # Document들 추가
    ######################################
    documents = load_and_split_olympic_data()
    vector_store.add_documents(documents=documents)

    return vector_store


def get_retriever(vector_store, k: int = 5):
    retriever = vector_store.as_retriever(
        search_kwargs={"k": k}
    )
    return retriever

In [ ]:
vectorstore = get_vectorstore()
basic_retriever = get_retriever(vectorstore) # naive rag (가장 기본)

In [7]:
basic_retriever.invoke("근대 올림픽은 언제 시작되었나요?")

[Document(metadata={'Header 1': '올림픽', '_id': '19f4ac63-0fc5-4753-9871-7e492970524a', '_collection_name': 'olympic_info_wiki'}, page_content='올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.  \n또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해

# Rerank

## 개념

- RAG의 정확도는 관련 정보의 컨텍스트 내 존재 유무가 아니라 순서가 중요하다. 즉, 관련 정보가 컨텍스트 내 상위권에 위치하고 있을 때 좋은 답변을 얻을 수 있다는 뜻이다. 
- **Rerank**는 RAG 시스템에서 **초기 검색 단계에서 추출된 후보 문서들의 순위를 재조정**하는 기법이다. 
- 벡터 유사도 기반의 빠른 1차 검색 후, 보다 정밀한 모델(예: Cross-encoder, LLM 등)을 활용해 질문과 검색된 문서간의 의미론적 관련성을 평가하여, 실제로 답변 생성에 가장 **적합한 문서들이 상위**에 오도록 순서를 다시 매긴다. 이를 통해 LLM이 더 정확하고 관련성 높은 정보를 바탕으로 답변을 생성할 수 있게 도와준다.

## 방법

- **Cross-encoder 기반 Rerank**  
  - Cross Encoder를 이용해서 순위를 재 지정한다.
  - Cross-encoder
    - 질문과 문서를 같이 입력으로 받아 둘간의 유사도 점수를 예측하도록 학습한 모델.
    - 학습을 두 문장의 유사도록 예측하도록 학습하였기 때문에 단순 유사도 검사 보다 두 문장간의 의미적 관련성등을 이용해 유사도를 예측하기 때문에 더 정확한 결과를 보인다.
  - 1차적으로 검색한 문서와 질문간의 유사도를 **cross-encoder**로 다시 계산해서 문서의 순위를 재 조정한다.
  
  > - **Bi-encoder**
  >     - 질문 (query)와 문서(document)를 각각 독립적으로 인코딩한 후, 벡터 간 유사도 계산
  >     - Encoder 모델은 개별 문장을 입력받아 embedding vector를 출력한다. 질문과 문서를 각각 encoding한 뒤에 둘 간의 유사도를 계산한다.
  
  ![bi_crosss_encoder](figures/bi_cross_encoder.png)

  \[출처:https://aws.amazon.com/ko/blogs/tech/korean-reranker-rag/\]

- **LLM 기반 Rerank**  
  GPT-3, GPT-4 등 LLM을 활용해 각 문서가 질문에 얼마나 부합하는지 평가하여 순위를 매긴다. 성능이 뛰어나지만 비용이 높고 응답 속도가 느릴 수 있다.
## Rerank RAG 프로세스  
1. 1차 검색(예: 임베딩 기반 벡터 검색)으로 상위 k개 문서 추출.  
2. Reranker 모델에 질문-문서 쌍을 입력.  
3. 각 쌍의 관련성 점수 산출 및 재정렬.  
4. 상위 n개 문서를 LLM의 컨텍스트로 전달하여 답변 생성.

## 장단점

- **장점**  
  - Rerank를 적용하면 단순 벡터 유사도 기반 검색보다 훨씬 정교하게 질문과 관련된 정보를 추출할 수 있다. 
  - 실제로 생성되는 답변의 품질이 크게 향상되며, 도메인 특화 정보나 복잡한 질의에도 높은 정확도를 보인다.

- **단점**  
  - Cross-encoder나 LLM 기반 Rerank는 연산량이 많아 실시간 응답이 필요한 대규모 서비스에선 속도 저하가 발생할 수 있다.
  - 초기 검색 결과(상위 k개)에만 적용하므로, 1차 검색의 품질이 낮으면 Rerank 효과가 제한적일 수 있다.

## CrossEncoder Reranker 예제

In [8]:
# 검색된 문서들을 질문을 바탕으로 압축 해서 context 를 재생성하는 Retrivever
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import CrossEncoderReranker
#CrossEncoder 를 HuggingfaceHub에서 가져온다. - 검색 : NLP> textrangking > 모델 이름에 rerangker 붙은 모델들
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

retriever = get_retriever(vectorstore, k = 10) # Rerank 할 때는 검색 개수를 크게 가져온다.

reranker_model = "BAAI/bge-reranker-v2-m3"
reranker = HuggingFaceCrossEncoder(model_name=reranker_model) # HUgging Face Reranker 모델 생성
compressor = CrossEncoderReranker(model=reranker, top_n=5) # Compressor 생성(context 를 줄이는 방법)
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever, # 기본 리트리버 설정 -> 이것으로 k(10)개 가져온다.
    base_compressor=compressor # 기본 리트리버가 가져온 문서를 압축하는 모델 (Renranker) 설정.  여기서 top_n 개 추출
)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

c:\Users\USER\Documents\SKN21\10_langchain\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to 

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [9]:
query = "올림픽에 발생한 다양한 논란들에 대해 정리해줘"
base_result = basic_retriever.invoke(query)
rerank_result = compression_retriever.invoke(query)


In [10]:
base_result_str=[doc.page_content for doc in base_result]
rerank_result_str = [doc.page_content for doc in rerank_result]

In [11]:
base_result_str

['올림픽 브랜드의 판매는 항상 논란이 되어왔다. 비판 중 하나는 올림픽이 지나치게 상업성과 연계되어 이제 올림픽이 기존의 상업적인 스포츠쇼와 다를 게 없다는 것이다. 1996년과 2000년 하계 올림픽 기간 사이에는 올림픽 관련 상품 시장의 포화 상태가 일어나면서 IOC에게 또 다른 비판이 일었다. 개최도시가 올림픽 관련 물건들을 파려는 상인과 회사들로 넘쳐났던 것이다. IOC는 이에 대해 앞으로 다가올 올림픽에서는 과다 경쟁을 방지하고 이런 불미스라운 상황이 일어나지 않도록 대처하겠다고 응답했다. 또 다른 비판으로는, 개최국이나 개최도시는 올림픽에 모든 비용을 들이는데 IOC는 손도 까딱 않고 올림픽 상징으로부터 얻는 모든 권리와 수입을 독차지하고 있다는 점이다. IOC는 올림픽 상징으로 인한 수입 뿐 아니라 스폰서와 중계권에서 들어오는 수입의 일정 지분도 가져간다. 개최도시 입장에서는 그들의 투자가 흑자가 될지 확신하지도 못하는 상황에서 올림픽 개최도시로서의 권리를 위해 노력을 해야 한다.',
 '올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해 참가를 거부했다. 캄보디아, 이집트, 이라크, 레바논은 제2차 중동 전쟁 때문에 보이콧했다. 1972년과 1976년 올림픽에는 많은 아프리카의 국가들이 남아프리카 공화국과 로디지아에서 일어나는 인종 차별정권에 대한 항의의 표시로 올림픽 참가를 거부했다. 이 보이콧에는 뉴질랜드도 관계가 되어있는데, 뉴질랜드 럭비 국가 대표팀이 당시 아파르트헤이트정책을 쓰던 남아프리카 공화국과 경기를 했음에도 불구하고 뉴질랜드의 올림픽 참가가 허용되었기 때문이었다. 국제 올림픽 위원회는 이 두 보이콧에 대해 심각하게 고민했으나 후자의 뉴질랜드의 경우는 럭비가 올림픽 종목이 아니라는 이유를 내세워 뉴질랜드의 올림픽 참가 금지 요청을 거부했다. 당시 아프리카에 속해 있던 20개국과 가이아나, 이라크는 경기를 끝낸 선수들이 있었지만 탄자니아가 이끄는 올림픽 보이콧

In [12]:
rerank_result_str

['올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었다. 그리고 19세기 말에 피에르 드 쿠베르탱 남작이 고대 올림피아 제전에서 영감을 얻어, 근대 올림픽을 부활시켰다. 이를 위해 쿠베르탱 남작은 1894년에 IOC를 창설했으며, 2년 뒤인 1896년에 그리스 아테네에서 제 1회 올림픽이 열렸다. 이때부터 IOC는 올림픽 운동의 감독 기구가 되었으며, 조직과 활동은 올림픽 헌장을 따른다. 오늘날 전 세계 대부분의 국가에서 올림픽 메달은 매우 큰 영예이며, 특히 올림픽 금메달리스트는 국가 영웅급의 대우를 받으며 스포츠 스타가 된다. 국가별로 올림픽 메달리스트들에게 지급하는 포상금도 크다. 대부분의 인기있는 종목들이나 일상에서 쉽게 접하고 즐길 수 있는 생활스포츠 종목들이 올림픽이라는 한 대회에서 동시에 열리고, 전 세계 대부분의 국가 출신의 선수들이 참여하는 만큼 전 세계 스포츠 팬들이 가장 많이 시청하는 이벤트이다. 2008 베이징 올림픽의 모든 종목 누적 시청자 수만 47억 명에 달하며, 이는 인류 역사상 가장 많은 수의 인구가 시청한 이벤트였다.  \n또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20세기의 변화하는 경제, 

# HyDE (Hypothetical Document Embedding)
## 개념
- 질문(query)에 대한 가상의 답변 문서를 생성하고, 이 생성된 가상의 답변 문서를 임베딩하여 검색에 활용하는 기법이다.
- 일반적인 RAG에서 검색은 질문을 임베딩하여 문서 임베딩과 직접 비교한다. 
- 질문("파리는 어떤 도시인가?")과 답변 문서("파리는 프랑스의 수도이며...")는 표현 방식이 다르다는 문제가 있다. 
- 즉 의미적으로는 관련있지만 벡터 공간(임베딩 벡터간의 유사서)에서는 거리가 멀 수가 있다.
- 그래서 HyDE는 질문과 문서가 아니라 질문으로 가상의 답변 문서를 만들고 **가상의 답변문서와 저장된 문서들간의 유사도**를 비교한다.

## HyDE 프로세스

1. 가상 문서 생성
   -  LLM을 사용해 질문에 대한 가상의 답변 문서 생성
   -  이때 성능이 좋은 LLM을 사용하는 것이 좋다.
2. 임베딩 변환
   - `1`에서 생성한 가상 문서를 벡터로 임베딩
3. 유사도 검색
   - 가상 문서 임베딩으로 Vector Store에 저장된 문서들 중 유사한 문서를 검색
4. 답변 생성
   - 검색된 실제 문서를 바탕으로 최종 답변 생성

## 장단점
- **장점**
  - 질문-문서 간 의미적 차이를 해결 해서 정확한 문서 검색 가능
  - 질문 표현 방식에 덜 민감
- **단점**
  - 가상 문서의 품질에 따른 성능 편차가 발생한다.
    - 가상 문서 생성 시 환각(hallucination) 위험
  - 추가적인 LLM 호출로 인한 비용 증가한다.

In [17]:
# HyDE chine(pipeline) 구성
models = ChatOpenAI(model="gpt-5-mini")
hyde_prompt = ChatPromptTemplate.from_template(
    template= """Instruction
다음 질문에 대한 완전하고 상세한 답변을 사실에 기반해서 작성해 주세요.
질문과 관련된 내용으로 답변을 작성합니다.
답변과 직접적인 관련성이 없는 내용은 절대 포함시키지 않습니다.

# 질문
{query}
"""
)


hyde_chain = hyde_prompt | models | StrOutputParser()

In [18]:
query = "올림픽에 발생한 다양한 논란들에 대해 정리해줘."
dummy_result = hyde_chain.invoke({"query":query})
dummy_result

'아래는 올림픽 역사에서 실제로 발생한 주요 논란들을 범주별로 정리한 내용입니다. 각 항목에는 대표적인 사례(연도·대회)와 핵심 쟁점을 간략히 설명했습니다.\n\n1) 도핑·약물복용 관련 논란\n- 벤 존슨 사건(1988 서울) — 남자 100m 금메달 획득 직후 스테로이드(스타노졸롤) 양성 반응으로 금메달 박탈. 육상 도핑 문제를 전 세계에 부각시킴.\n- 마리온 존스(2000 시드니) — 이후 BALCO 등 불법 약물 연루를 자백하고 여러 메달을 반납, 관련 진술로 법적 책임을 짐.\n- 동독의 국가 주도 도핑(1970s–80s) — 여자 선수들을 중심으로 조직적·비밀리에 아나볼릭 스테로이드 투여가 이루어졌음. 많은 선수들이 건강 문제를 호소.\n- 러시아 국가 주도 도핑(특히 2014 소치 관련, 2016 McLaren 보고서) — 조직적 샘플 조작과 은닉이 확인되어 러시아 선수단에 부분·전면 출전 제한 및 메달 박탈 등 제재가 가해짐(평창 2018 일부 선수는 OAR 명칭으로 출전).\n- 재검사로 인한 메달 박탈·재배분 — IOC가 보관 샘플을 재검사하면서 2008·2012 등 과거 대회의 다수 메달이 소급 박탈되고 수여 순위가 변경된 사례가 잦음.\n\n2) 심판·판정 논란\n- 피겨스케이팅 커플 경기(2002 솔트레이크시티) — 러시아 쌍과 캐나다 쌍의 채점에서 프랑스 심판의 부정 개입 시인으로 대회 직후 파문, 결국 두 쌍에 공동 금메달 수여.\n- 체조 남자 개인종합(2004 아테네) — 한국 선수 양태영의 평행봉 출발점수(start value) 채점 실수 논란으로 미국의 폴 햄이 금메달을 차지했으며 이의 제기가 받아들여지지 않아 논쟁이 지속됨.\n- 복싱 점수·판정 논란(예: 1988 서울 로이 존스 주니어 결승) — 경기 지배에도 판정에서 패하는 사례들이 있어 시스템 전반의 공정성 문제 제기(채점 방식 개편으로 이어짐).\n- 심판·채점 시스템의 불투명성 문제 — 여러 종목에서 판정 기준·전자채점 시스템·인간 심판의 주관성 등으로 논란이 반

In [27]:
#hyde 를 이용한 RAG Chain 구성
#최종 응답을 요청하는 Prompt
prompt = ChatPromptTemplate.from_template(
    template=""" #Instruction
당신은 올림픽 전문가 입니다.
제공된 Context를 바탕으로 질문에 답변을 합니다.
만약 Context에 질문과 관련된 내용이 없으면 "정확한 정보가 없어서 답을 알 수가 없습니다."라고 대답합니다.
Context에 없는 내용을 답변에 포함시키지 않습니다.

# 질문
{query}

# Context
{context}

#Output Indicator
-답변을 만들 때 참조한 context의 내용을 같이 출력합니다.
-참조 내용은 각주 형식으로 작성합니다.

## 출력 예:
답변 내용[1] 답변 내용[2]

[1] 참조내용1
[2] 참조내용2
"""
)

def format_str(document_list: list) -> str:
    #list[Document] - Document의 page_content 만 추출해서 반환.
    return '\n\n'.join(doc.page_content for doc in document_list)

# chain -> {context:hyde_chain | retriver, query: 질문}
chain = {
    "query":RunnablePassthrough(),
    "context": hyde_chain | basic_retriever | format_str # 질문(query)-(hyde_chain)-> 가상답변->Retrieve한 결과
} | prompt | models | StrOutputParser()

In [20]:
result = chain.invoke(query)

In [21]:
print(result)

아래는 제공된 Context에 근거한 올림픽 관련 주요 논란들의 정리입니다.

- 도핑 및 약물 복용 문제
  - 20세기 초반부터 선수들이 기록 향상을 위해 약물을 복용하기 시작했고(예: 1904년 마라톤 우승자 토머스 J. 힉스), 약물 과다 복용으로 사망한 사례도 있었다. 1960년 로마 대회에서 덴마크 사이클선수 크누드 에네마르크 옌센의 사망을 검시에서 암페타민 과다복용으로 결론내렸다. 이후 1960년대 중반부터 경기 연맹들이 금지 조치를 취했고 1967년 IOC도 약물 복용 금지에 동참했다[1].
  - 올림픽에서 도핑 양성으로 메달을 박탈당한 초기 사례로 1968년의 한스 군나르 리렌바르(근대5종 동메달 박탈, 음주 양성 반응)가 있고, 가장 유명한 사례는 1988년 육상 100m 금메달리스트 벤 존슨(스타노졸롤 복용으로 금메달 박탈, 금메달이 칼 루이스에게 넘어감)이다[2].
  - 1999년 세계반도핑기구(WADA)가 설립되었고 2000·2002년 대회에서 약물 양성 반응 선수가 급증했으며(역도·크로스컨트리 등에서 실격), 2006년 동계올림픽에서는 메달리스트 1명이 양성반응으로 메달을 반납해야 했다. IOC의 약물 판정 기준이 인정받았고, 2008년 베이징 대회 기간중 3,667명의 선수가 검사를 받았으며 몇몇 선수는 대회 전 NOC에 의해 출전금지 조치를 당했고 대회 기간중에는 단 3명만 도핑 검사에 걸렸다[3].

- 정치적 이용 및 선수의 정치적 표현
  - 1936년 베를린 대회에서 나치 독일은 올림픽을 자국 선전의 장으로 이용하려 했지만 제시 오언스의 금메달 4개로 계획은 좌절되었다[4].
  - 소련은 1952년 헬싱키 대회부터 올림픽에 참가했고, 그 전에는 1928년부터 스파르타키아다에 참가하는 등 이념적·체제적 대항을 보여주었다(공산권의 노동자 올림픽 등)[4].
  - 1968년 멕시코시티 대회에서 토미 스미스와 존 카를로스가 시상식에서 블랙 파워 경례를 했고, 피터 노먼은 이들을 지지했다. 이에 IOC 위원장 에이버리 브런디지

#  MultiQueryRetriever

## 개념
- 하나의 사용자 질문으로 **여러 개의 다양한 질문을 생성하여 검색**을 수행하는 방법이다.
- 단일 질문의 한계를 극복하고 다각도에서 관련 정보 검색할 수있다.
  - 기본 RAG는 사용자의 질문의 질(quality)에 따라 검색 결과가 좌우된다.
  - 사용자가 한 질문에만 의존하는 것이 아니라 그 질문을 바탕으로 **다양한 의미의 질문들을 생성해서 단일 질문이 가지는 표현의 한계를 보완**한다.
    - 동일한 질문을 다른 각도에서 접근할 수있다.
    - 다양한 어휘와 표현으로 질문을 재구성한다.
- 예)
  - **원본 질문**: "딥러닝의 장점은 무엇인가?"
  - **생성된 질문들**:
    1. "딥러닝이 전통적인 머신러닝보다 나은 점은?"
    2. "딥러닝을 사용하면 얻을 수 있는 이익은?"
    3. "딥러닝의 주요 강점과 특징은?"
    4. "딥러닝 기술의 핵심 우위는?"
## 실행 프로세스

1. 질문 생성   
   - LLM을 사용해 원본 질문을 3-5개의 서로 다른 질문으로 변환
2. 병렬 검색
   - 생성된 각 질문으로 독립적으로 문서 검색 수행
3. 결과 통합
   - 여러 검색 결과를 하나로 병합
4. 중복 제거
   - 동일한 문서가 여러 번 검색된 경우 중복 제거
5. 최종 답변
   - 통합된 문서 세트를 바탕으로 답변 생성

## 장단점
- **장점**
    - 단일 질문으로 놓칠 수 있는 관련 문서 발견 수 있다.
    - 사용자 질문 표현 방식의 한계 극복
    - 더 포괄적이고 완전한 정보 검색 및 수집을 할 수있다.
- **단점**
    - 여러 번의 LLM 호출과 검색 수행이 실행 되므로 **계산비용, 토큰비용, 응답시간이 증가한다.**
    - 생성된 질문의 품질에 따른 성능 편차가 있을 수 있다.
    - 생성된 질문에 따라 원래 질문과 관련성 낮은 문서도 검색될 수 있어 최종 답변을 방해하는 노이즈가 증가할 수있다.

In [23]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

mq_retriever = MultiQueryRetriever.from_llm(
    retriever=basic_retriever, # 질문들을 생성한 뒤 문서를 검색할 retriever
    llm=ChatOpenAI(model="gpt-5-nano"), # 여러가지 질문을 생성할 llm 모델
    # prompt="질문을 만들 때 사용할 Prompt Templet을 직접 넣을 수 있다."
)

In [24]:
dummy_querys = mq_retriever.invoke(query)
dummy_querys

[Document(metadata={'Header 1': '올림픽', 'Header 2': '논란', 'Header 3': '약물 복용', '_id': 'c947b375-6b9f-4876-b3bb-c3d9a43ddd7a', '_collection_name': 'olympic_info_wiki'}, page_content='20세기 초반, 많은 운동 선수들은 기록향상을 위해 약물을 복용하기 시작했다. 예를 들어 1904년 하계 올림픽 마라톤에서 우승한 미국 선수 토머스 J. 힉스는 코치에게서 스트리크닌과 브랜디를 받았다. 올림픽에서 약물을 과다 복용으로 사망한 사례도 한 번 있었다. 1960년 로마 대회 때 사이클 개인도로 경기 중에 덴마크 선수인 크누드 에네마르크 옌센이 자전거에서 떨어져서 사망했다. 검시관들의 조사에 의하면 그의 죽음의 원인은 암페타민 과다 복용이라고 했다. 이에 1960년대 중반부터 각 경기 연맹은 약물 복용을 금지하기 시작했으며 1967년에는 IOC도 약물 복용 금지에 동참했다.  \n올림픽에서 약물 복용 양성 반응이 나와서 메달을 박탈당한 첫 번째 사례로는 1968년 하계 올림픽의 근대 5종 경기에 출전해 동메달을 딴 한스 군나르 리렌바르가 있다. 그는 경기 후 도핑검사 결과 알코올을 복용한 것으로 확인되어 메달을 박탈당했다. 도핑 양성 반응으로 메달을 박탈당한 것으로 가장 유명한 사람은 1988년 하계 올림픽 육상 100m 경기에서 금메달을 땄으나 도핑 검사 결과 스타노졸롤을 복용한 것으로 확인돼 금메달을 박탈당한 캐나다 선수인 벤 존슨이 있다. 이에 따라 금메달은 2위를 했던 칼 루이스가 대신 받았다.  \n1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메

In [29]:
chain = {
    "context":mq_retriever | format_str,
    "query":RunnablePassthrough()
} | prompt | models | StrOutputParser()

response = chain.invoke(query)

In [30]:
print(response)

아래는 제공된 Context에 근거한 올림픽에서 제기된 주요 논란들 정리입니다.

- 도핑(약물 복용) 및 약물 관련 사고와 메달 박탈: 근대 초부터 선수들의 약물 복용 사례가 있었고(예: 1904년 마라톤 우승자 토머스 J. 힉스), 1960년 로마 대회에서는 덴마크 사이클 선수의 약물 과다 복용으로 인한 사망 사례가 있었으며, 1967년 IOC가 약물 복용 금지에 동참했다. 도핑 양성으로 인한 메달 박탈 사례(예: 1968년 한스 군나르의 동메달 박탈, 1988년 벤 존슨의 금메달 박탈)와 1999년 WADA 설립 이후에도 2000·2002·2006·2008 대회 등에서 도핑 양성 반응 사례들이 보고되었다[1][2][3].

- 정치적 이용·표현과 보이콧 등: 올림픽이 정치나 체제 선전의 장으로 이용된 사례(예: 1936년 나치독일의 선전 시도), 냉전 시기 강대국들의 이데올로기 선전 활용과 소련의 참가(1952년부터), 그리고 선수 개인의 정치적 표현(1968년 멕시코시티의 토미 스미스·존 카를로스의 블랙 파워 경례) 등 정치 관련 논란이 있었다. 또한 Context에서는 ‘보이콧’이 올림픽과 관련된 문제들 가운데 하나로 언급되어 있다[4][5][6].

- 국가 간 정치 갈등에 따른 경기 회피 사례: 일부 국가는 특정 상대국과의 대결을 회피하는 경우가 있으며, 예로 이란이 이스라엘 선수와의 경기를 피하는 사례들이 언급되어 있다[7].

- IOC 내부 부패·뇌물 및 유치과정의 비리: 1998년에 일부 IOC 위원들이 2002 솔트레이크시티 유치 과정에서 뇌물 관련 폭로를 당했고, 이로 인해 위원 사퇴 및 퇴출과 조사가 이루어졌다. 또한 BBC 다큐멘터리(2004)는 2012년 개최지 선정 관련 뇌물 의혹을 제기했고, 토리노 유치 관련 의혹들도 제기되었다[8][9][10].

- IOC 지도자들에 대한 비판(정책·편향·부패 관련): 에이버리 브런디지와 후안 안토니오 사마란치 같은 전·현직 IOC 위원장들이 정치적 편향이나 족벌 정치·부패 협력 의혹 등으로

# MapReduce RAG 방식

## 개요

- RAG(Retrieval-Augmented Generation)에서 검색된 문서들 중 질문과 관련성이 높은 문서만을 선별하여 더 정확한 답변을 생성하는 방법이다.
- 검색된 문서들 중에서 질문 답변에 실제로 도움이 되는 문서만을 LLM을 통해 선별한 후 전달하는 방식이다.

## MapReduce 방식 프로세스
1. Map (문서 검색)
   - 벡터스토어에서 질문과 유사한 문서들을 의미적 유사도 검색으로 찾는다.
   - 이 단계에서는 단순 벡터 유사도만 고려하므로 질문과 직접적인 관련이 없는 문서도 포함될 수 있다.
2. Reduce (문서 선별 및 요약)
   - 검색된 각 문서가 질문 답변에 실제로 도움이 되는지 LLM에게 평가 요청한다.
   - 관련성이 높은 문서들만 선별하여 요약하거나 결합한다.
   - 필요시 여러 문서의 정보를 통합하여 더 응답에 적합한 컨텍스트를 생성한다.
3.  Generate (최종 답변 생성)
    - 질문과 선별된 컨텍스트를 함께 LLM에 전달하여 최종 답변을 생성한다.

## 장단점

- **장점**
  - **높은 정확도**: 질문과 직접 관련된 정보만 사용하여 더 정확한 답변을 생성한다.
  - **노이즈 제거**: 유사하지만 관련 없는 정보로 인한 혼동을 방지한다.
  - **컨텍스트 최적화**: 제한된 토큰 범위 내에서 가장 유용한 정보만 전달한다.
  - **확장성**: 많은 문서가 검색되어도 중요한 정보만 선별하여 처리할 수 있다.
- 단점
  - **추가 비용**: 문서 선별을 위한 LLM 호출로 인한 비용이 증가한다.
  - **처리 시간**: 문서 평가 단계가 추가되어 응답 속도가 저하된다.
  - **복잡성**: 구현과 관리가 더 복잡하다.
  - **의존성**: 문서 선별 성능이 LLM의 판단 능력에 크게 의존한다.

In [38]:
#reduce : 질문과 문서 간의 직접적 연관이 있는지를 판단해서 연관있는 문서만 반환

reduce_prompt = ChatPromptTemplate.from_template(
    template=""" #Instruction
Context의 내용이 질문과 관련성이 있으면 입력된 context 를 그대로 반환하고 관련성이 없으면 아무것도 출력하지 않는다.
- 관련성 판단 기준
    -Context에 질문에 대한 직접적인 답이나 답을 위한 단서가 있어야 한다.
    -Context의 정보가 질문을 해결하는 데 도움을 줘야 한다.

#Context
{context}

#질문
{query}

#Output Indicator
-질문과 Context가 환련이 있으면 context의 내용만 정확히 그대로 반환한다. 어떠한 내용도 추가히지 않는다.
-질문과 Context가 관련이 없으면 아무것도 출력하지 않는다.
"""
)

reduce_chain = reduce_prompt | ChatOpenAI(model="gpt-5-mini") | StrOutputParser()

In [39]:
# 테스트
reduce_chain.invoke({"context":"사과는 과일입니다.", "query": " 올림픽 종목에 대해 알려줘"})

''

In [44]:
from langchain_core.runnables import chain

@chain
def map_reduce(inputs:dict) -> str:
    """
    Retriever 가 조회한 Document 들과 사용자 질문을 받아서 질문과 개별 문서 간의 관련성을 reduce_chain 을 이용해 검사한다.
    document 들 중 관련성 있는 문서들만 추려서 반환한다.

    args:
        inputs(dict) - dict[list[document], query:str] {Retriever가 조회한 문서들, 사용자 질문}
    """

    docs = inputs['documents']
    query = inputs['query']
    contexts = "" #질문과 관련있는 문서들 모을 str
    for doc in docs:
        res = reduce_chain.invoke({"context":doc.page_content, "query":query})
        if res.strip() != None:
            contexts += res+"\n\n"

    return contexts.strip()

retriever = get_retriever(vectorstore, k=10) # k는 크게 해서 많이 조회한다.
map_reduce_chain = {
    "documents": retriever, "query":RunnablePassthrough()
} | map_reduce

In [45]:
map_reduce_chain.invoke("국제 올림픽 기구에 대해 설명해 줘.")

'올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다.국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.  \n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)\n- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.  \n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.\n\n올림픽에서 첫 번째 보이콧은 1956년 하계 올림픽에서 시작되었다. 네덜란드, 스페인, 스위스는 소련의 헝가리 침공에 항의해 참가를 거부했다. 캄보디아, 이집트, 이라크, 레바논은 제2차 중동 전쟁 때문에 보이콧했다. 1972년과 1976년 올림픽에는 많은 아프리카의 국가들이 남아프리카 공화국과 로디지아에서 일어나는 인종 차별정권에 대한 항의의 표시로 올림픽 참가를 

In [47]:
prompt = ChatPromptTemplate.from_template(
    template=""" #Instruction
당신은 올림픽 전문가 입니다.
제공된 Context를 바탕으로 질문에 답변을 합니다.
만약 Context에 질문과 관련된 내용이 없으면 "정확한 정보가 없어서 답을 알 수가 없습니다."라고 대답합니다.
Context에 없는 내용을 답변에 포함시키지 않습니다.

# 질문
{query}

# Context
{context}

#Output Indicator
-답변을 만들 때 참조한 context의 내용을 같이 출력합니다.
-참조 내용은 각주 형식으로 작성합니다.

## 출력 예:
답변 내용[1] 답변 내용[2]

[1] 참조내용1
[2] 참조내용2
"""
)
model = ChatOpenAI(model="gpt-5-mini")
chain = {
    "context":map_reduce_chain, "query":RunnablePassthrough()
} | prompt | model | StrOutputParser()

In [48]:
response = chain.invoke("올림픽 경기 종목에 대해 설명해줘")


# Self Query Retriever

- **Self-Query Retriever**는 사용자의 자연어 질문을 LLM을 통해 '의미 기반 검색 쿼리(semantic query)'와 '메타데이터 기반 필터 조건(metadata filter)'으로 자동 분해/구조화하여, 벡터 검색과 조건 기반 필터링을 동시에 수행하도록 설계된 Advanced RAG의 고급 Retriever 기법이다. 이를 통해 메타데이터의 속성을 정확히 반영한 정밀 검색이 가능하다.
- 즉 사용자 질의로 부터 메타데이터의 필터 조회시 사용할 값을 추출하여 metadata 조건 기반 필터링을 할 수 있도록 한다.

## Self Query Retriever의 구성 요소

- **LLM (Large Language Model)**
    - 자연어로 표현된 사용자 질문을 받아, 이를 메타데이터 조건과 필터링 조건이 포함된 정형 쿼리(Structured Query)로 변환하는 역할을 한다.
- **StructuredQuery**
    - 사용자의 질문을 기반으로 생성되는 구조화된 쿼리로 다음 두가지가 생성된다.
      - 벡터 유사도 검색을 위한 **의미 기반 검색 쿼리**(semantic query)
      - 문서 메타데이터를 이용한 **필터링 조건**(metadata filter)
  
- **Query Translator**
    - 생성된 StructuredQuery를 특정 벡터 데이터베이스(Qdrant, Chroma 등)의 쿼리 언어로 번역하여, 실제 검색이 가능하도록 한다.
- **Vector Database**
    - 변환된 쿼리를 바탕으로 벡터 유사도 검색과 메타데이터 조건 필터링을 함께 수행하여 관련 문서를 반환한다.

## Self Query Retriever 작동 원리

![selfquery retriever](figures/selfquery_retriever.jpg)

1. 사용자가 자연어 질의(Query)를 입력한다.
2. LLM이 입력된 자연어 질문을 해석하여 **Query Constructor**가 **StructuredQuery**로 변환한다.
3. **Query Translator**가 StructuredQuery를 **벡터 데이터베이스에서 이해할 수 있는 쿼리**로 변환한다.
4. 벡터 데이터베이스가 변환된 쿼리에 따라 문서를 검색한다.
5. 최종적으로, 검색 결과가 사용자에게 제공된다.

## 사용 예시
1. Query
    - "2023년에 발표된 OpenAI의 GPT 모델 관련 논문을 찾아줘."
2. Query Constructor가 위 질문을 다음과 같은 형태의 StructuredQuery로 변환한다.

    ```json
    {
        "query": "GPT 모델 논문",
        "filter": {
            "must": [
                {
                    "key": "year",
                    "match": {
                        "value": 2023
                    }
                },
                {
                    "key": "author",
                    "match": {
                        "value": "OpenAI"
                    }
                }
            ]
        }
    }
    ```
     - query: 벡터검색에 사용될 자연어 의미기반 쿼리(semantic query)
     - filter: 메타데이터 기반 필터 조건
3. 위의 StruncturedQuery는 Retriever와 연결된 Vector database의 검색 형식에 맞춰 query translator에 의해 변환 되고 이것을 이용해 검색을 진행한다. (형식은 DB 마다 다르다.)


## Self Query Retriever의 장점

- **정밀성**: 메타데이터 조건을 정확하게 지정하여 원하는 문서를 정밀하게 검색할 수 있다.
- **효율성**: 메타데이터 필터링을 통해 관련 없는 문서를 미리 제거하여 검색 대상 문서의 후보 집합을 사전에 축소함으로써, 벡터 유사도 계산 대상이 줄어들고 전체 검색 연산량을 감소시킬 수 있다.
- **사용자 편의성**: 사용자가 복잡한 쿼리 조건을 직접 작성하지 않고, 자연어로 간편하게 질문할 수 있다.

## 예제
- 필요 패키지 설치
  - pip install langchain langchain-community langchain-openai langchain-qdrant lark

> ### Lark 패키지
> 
> - Lark는 텍스트를 구조적으로 분석하기 위한 파싱 라이브러리로, 미리 정의한 문법에 따라 문장을 해석하여 정해진 형태(parse tree 또는 abstract syntax tree) 형태의 구조화된 결과를 생성한다.
> - 이 라이브러리는 컴파일러, 인터프리터, DSL(도메인 특화 언어), 쿼리 언어, 수식 해석기처럼 구조화된 입력을 처리해야 하는 다양한 프로그램에서 활용된다.
> - LangChain의 **SelfQueryRetriever**에서는 LLM이 생성한 쿼리 조건 문자열(output)을 석하여 메타데이터 필터 조건으로 변환하기 위해 Lark가 사용된다. Lark는 텍스트 형식의 조건을 분석하여 벡터 검색 시 사용되는 StructuredQuery의 filter(metadata filter) 구조로 변환해주는 역할을 담당한다.

In [49]:
!uv pip install lark

Resolved 1 package in 54ms
Installed 1 package in 63ms
 + lark==1.3.1


In [1]:
from langchain_qdrant import QdrantVectorStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
documents = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

In [3]:
# Vector Store 생성

COLLECTION_NAME = "example"
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(host="localhost", port=6333)
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

vectorstore = QdrantVectorStore(
    client=client,
    collection_name="example",
    embedding=embedding_model
)
vectorstore.add_documents(documents=documents)

['a2e2bf59ddfd429eb96b9b35719d11aa',
 '0290525beeec4ae3b6ea87d5d4f14878',
 '5b4dae57c3834dd780f8f979c17208d0',
 '03d4f7fb9e2541a9ac6613ca0085a6fd',
 '5ea44f123a87482198d9d6fa4da52ff3',
 '6e3b16cd7304457a900b5c56fb8d6e6b']

In [11]:
# selfQueryRetriever 생성
# 질문에서 추출할 Metadata 설계를 위한 클래스. 개별 Field 선언 할 때 사용됨 (attributeinfo)
from langchain_classic.chains.query_constructor.schema import AttributeInfo
from langchain_classic.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# 추출할 Metadata(payload) 정의 => AttributeInfo: name(메타데이터 이름), discription: 메타데이터 설명, type: 데이터 타입
metadata_fields_schema = [
    AttributeInfo (
        name="genre",
        description="영화장르. 다음 중 하나를 선택한다. ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']", 
        # 범주형일 경우 범주값 지정
        type="string"
    ),
    AttributeInfo(
        name="year", description="영화 개봉 년도", type="integer"
    ),
    AttributeInfo(
        name="director", description="영화를 만든 감독의 영문 이름.", type="string"
    ),
    AttributeInfo(
        name="rating", description="영화 평점. 1~10 사이의 실수 값을 가진다.", type="float"
    )
]

# 문서에 어떤 내용들이 있는지 설명
document_description = "영화 내용에 대한 짧은 소개"

# 쿼리 생성기(Query Contructor) -> LLM 모델
llm = ChatOpenAI(model="gpt-5-mini")

#SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents=document_description,
    metadata_field_info=metadata_fields_schema
)

In [13]:
# docs = retriever.invoke("평점이 8점 이상인 영화를 보고 싶어.")
docs = retriever.invoke("2000년 이후에 개봉한 영화를 추천해줘.")

for doc in docs:
    print(doc.metadata)

{'year': 2019, 'director': 'Greta Gerwig', 'rating': 8.3, '_id': '03d4f7fb-9e25-41a9-ac66-13ca0085a6fd', '_collection_name': 'example'}
{'year': 2010, 'director': 'Christopher Nolan', 'rating': 8.2, '_id': '0290525b-eeec-4ae3-b6ea-87d5d4f14878', '_collection_name': 'example'}
{'year': 2006, 'director': 'Satoshi Kon', 'rating': 8.6, '_id': '5b4dae57-c383-4dd7-80f8-f979c17208d0', '_collection_name': 'example'}


In [ ]:
# retriever + query -> 응답 prompt -> llm -> ouput paser (chain 구성)